In [1]:
AUTHOR_NAME = 'Jane Deijnen'
AUTHOR_ID_NR = '1354396'
AUTHOR_DATE = '2020-05-05'

AUTHOR_NAME, AUTHOR_ID_NR, AUTHOR_DATE

('Jane Deijnen', '1354396', '2020-05-05')

In [2]:
import numpy as np
import pandas as pd
import bokeh as bk
import random

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import seaborn as sns
sns.set()  # set Seaborn defaults
plt.rcParams['figure.figsize'] = 10, 5  # default hor./vert. size of plots, in inches
plt.rcParams['lines.markeredgewidth'] = 1  # to fix issue with seaborn box plots; needed after import seaborn
from sklearn.cluster import KMeans  # for clustering

from bokeh.io import output_notebook, show, reset_output, curdoc
from bokeh.models import Slider, CustomJS, Select
from bokeh.plotting import figure
from bokeh.layouts import layout, column
from bokeh.models import (
    ColumnDataSource, Div,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
    FactorRange,
    ImageURL
)
from bokeh.palettes import BuPu
from bokeh.palettes import Colorblind8
output_notebook()

Loading BokehJS ...

In [3]:
#import eye tracking data
Eyetracking_data = pd.read_csv('metro_data.csv', encoding = 'latin1', sep = ";")
Eyetracking_data.head()

,Timestamp,StimuliName,FixationIndex,FixationDuration,MappedFixationPointX,MappedFixationPointY,user,description
0,2586,01_Antwerpen_S1.jpg,9,250,1151,458,p1,color
1,2836,01_Antwerpen_S1.jpg,10,150,1371,316,p1,color
2,2986,01_Antwerpen_S1.jpg,11,283,1342,287,p1,color
3,3269,01_Antwerpen_S1.jpg,12,433,762,303,p1,color
4,3702,01_Antwerpen_S1.jpg,13,183,624,297,p1,color


In [4]:
#replacing weird coding fails in stimuli names
Eyetracking_data['StimuliName'] = Eyetracking_data['StimuliName'].replace({'24_Z?rich_S1.jpg': '24_Zurich_S1.jpg', '24_Zrich_S1.jpg' : '24_Zurich_S1.jpg',
                                                                          '24_Zrich_S2.jpg' : '24_Zurich_S2.jpg', '24_Z?rich_S2.jpg' : '24_Zurich_S2.jpg',
                                                                          '24b_Z?rich_S1.jpg' : '24b_Zurich_S1.jpg', '24b_Zrich_S1.jpg' : '24b_Zurich_S1.jpg',
                                                                          '24b_Z?rich_S2.jpg' : '24b_Zurich_S2.jpg', '24b_Zrich_S2.jpg' : '24b_Zurich_S2.jpg', 
                                                                           
                                                                           '12_Br?ssel_S1.jpg' : '12_Brussel_S1.jpg', '12_Brssel_S1.jpg' : '12_Brussel_S1.jpg',
                                                                          '12_Br?ssel_S2.jpg' : '12_Brussel_S2.jpg', '12_Brssel_S2.jpg' : '12_Brussel_S2.jpg',
                                                                          '12b_Br?ssel_S1.jpg' : '12b_Brussel_S1.jpg', '12b_Brssel_S1.jpg' : '12b_Brussel_S1.jpg',
                                                                          '12b_Br?ssel_S2.jpg' : '12b_Brussel_S2.jpg', '12b_Brssel_S2.jpg' : '12b_Brussel_S2.jpg',
                                                                           
                                                                          '14_D?sseldorf_S1.jpg' : '14_Dusseldorf_S1.jpg', '14_Dsseldorf_S1.jpg' : '14_Dusseldorf_S1.jpg',
                                                                          '14_D?sseldorf_S1.jpg' : '14_Dusseldorf_S2.jpg', '14_Dsseldorf_S2.jpg' : '14_Dusseldorf_S2.jpg',
                                                                          '14b_D?sseldorf_S1.jpg' : '14b_Dusseldorf_S1.jpg', '14b_Dsseldorf_S1.jpg' : '14b_Dusseldorf_S1.jpg',
                                                                          '14b_D?sseldorf_S2.jpg' : '14b_Dusseldorf_S2.jpg', '14b_Dsseldorf_S2.jpg' : '14b_Dusseldorf_S2.jpg',
                                                                           
                                                                          '15_G?teborg_S1.jpg' : '15_Goteborg_S1.jpg', '15_Gteborg_S1.jpg' : '15_Goteborg_S1.jpg',
                                                                          '15_G?teborg_S2.jpg' : '15_Goteborg_S2.jpg', '15_Gteborg_S2.jpg' : '15_Goteborg_S2.jpg',
                                                                          '15b_G?teborg_S1.jpg' : '15b_Goteborg_S1.jpg', '15b_Gteborg_S1.jpg' : '15b_Goteborg_S1.jpg',
                                                                          '15b_G?teborg_S2.jpg' : '15b_Goteborg_S2.jpg', '15b_Gteborg_S2.jpg' : '15b_Goteborg_S2.jpg',
                                                                          
                                                                          '04_K?ln_S1.jpg' : '04_Koln_S1.jpg', '04_Kln_S1.jpg' : '04_Koln_S1.jpg',
                                                                          '04_K?ln_S2.jpg' : '04_Koln_S2.jpg', '04_Kln_S1.jpg' : '04_Koln_S2.jpg',
                                                                           '04b_K?ln_S1.jpg' : '04b_Koln_S1.jpg', '04b_Kln_S1.jpg' : '04b_Koln_S1.jpg',
                                                                          '04b_K?ln_S2.jpg' : '04b_Koln_S2.jpg', '04b_Kln_S2.jpg' : '04b_Koln_S2.jpg',})

In [5]:
#Grouping data per map
Antwerpen_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '01_Antwerpen_S1.jpg']
Antwerpen_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '01_Antwerpen_S2.jpg']

Antwerpen_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '01b_Antwerpen_S1.jpg']
Antwerpen_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '01b_Antwerpen_S2.jpg']

Berlin_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '02_Berlin_S1.jpg']
Berlin_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '02_Berlin_S2.jpg']

Berlin_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '02b_Berlin_S1.jpg']
Berlin_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '02b_Berlin_S2.jpg']

Bordeaux_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '03_Bordeaux_S1.jpg']
Bordeaux_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '03_Bordeaux_S2.jpg']

Bordeaux_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '03b_Bordeaux_S1.jpg']
Bordeaux_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '03b_Bordeaux_S2.jpg']

Köln_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '04_Koln_S1.jpg']
Köln_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '04b_Koln_S1.jpg']
Köln_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '04b_Koln_S2.jpg']

Frankfurt_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '05_Frankfurt_S1.jpg']
Frankfurt_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '05_Frankfurt_S2.jpg']

Frankfurt_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '05b_Frankfurt_S1.jpg']
Frankfurt_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '05b_Frankfurt_S2.jpg']

Hamburg_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '06_Hamburg_S1.jpg']
Hamburg_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '06_Hamburg_S2.jpg']

Hamburg_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '06b_Hamburg_S1.jpg']
Hamburg_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '06b_Hamburg_S1.jpg']

Moskau_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '07_Moskau_S1.jpg']
Moskau_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '07_Moskau_S2.jpg']

Moskau_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '07b_Moskau_S1.jpg']
Moskau_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '07b_Moskau_S2.jpg']

Riga_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '08_Riga_S1.jpg']
Riga_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '08_Riga_S1.jpg']

Riga_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '08b_Riga_S1.jpg']
Riga_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '08b_Riga_S1.jpg']
Tokyo_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '09_Tokyo_S1.jpg']
Tokyo_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '09_Tokyo_S2.jpg']

Tokyo_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '09b_Tokyo_S1.jpg']
Tokyo_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '09b_Tokyo_S2.jpg']

Barcelona_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '10_Barcelona_S1.jpg']
Barcelona_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '10_Barcelona_S2.jpg']

Barcelona_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '10b_Barcelona_S1.jpg']
Barcelona_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '10b_Barcelona_S2.jpg']

Bologna_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '11_Bologna_S1.jpg']
Bologna_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '11_Bologna_S2.jpg']

Bologna_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '11b_Bologna_S1.jpg']
Bologna_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '11b_Bologna_S2.jpg']

Brüssel_S1  = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '12_Brüssel_S1.jpg']
Brüssel_S2  = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '12_Brüssel_S2.jpg']

Brüssel_S1b  = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '12b_Brüssel_S1.jpg']
Brüssel_S2b  = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '12b_Brüssel_S2.jpg']

Budapest_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '13_Budapest_S1.jpg']
Budapest_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '13_Budapest_S2.jpg']

Budapest_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '13b_Budapest_S1.jpg']
Budapest_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '13b_Budapest_S2.jpg']

Düsseldorf_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '14_Dusseldorf_S1.jpg']
Düsseldorf_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '14_Dusseldorf_S2.jpg']

Düsseldorf_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '14b_Dusseldorf_S1.jpg']
Düsseldorf_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '14b_Dusseldorf_S2.jpg']
                                      
Göteborg_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '15_Goteborg_S1.jpg']
Göteborg_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '15_Goteborg_S2.jpg']
                                      
Göteborg_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '15b_Göteborg_S1.jpg']
Göteborg_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '15b_Göteborg_S2.jpg'] 

Hong_Kong_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '16_Hong_Kong_S1.jpg']
Hong_Kong_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '16_Hong_Kong_S2.jpg'] 
                                      
Hong_Kong_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '16b_Hong_Kong_S1.jpg']
Hong_Kong_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '16b_Hong_Kong_S2.jpg'] 
                                      
Krakau_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '17_Krakau_S1.jpg']
Krakau_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '17_Krakau_S2.jpg']
                                      
Krakau_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '17b_Krakau_S1.jpg']
Krakau_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '17b_Krakau_S2.jpg']
                                      
Ljubljana_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '18_Ljubljana_S1.jpg']
Ljubljana_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '18_Ljubljana_S2.jpg']
                                      
Ljubljana_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '18_Ljubljana_S1.jpg']
Ljubljana_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '18_Ljubljana_S2.jpg']                                      

New_York_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '19_New_York_S1.jpg']
New_York_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '19_New_York_S2.jpg']

New_York_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '19b_New_York_S1.jpg']
New_York_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '19b_New_York_S2.jpg']


Paris_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '20_Paris_S1.jpg']
Paris_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '20_Paris_S2.jpg']

Paris_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '20b_Paris_S1.jpg']
Paris_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '20b_Paris_S2.jpg']

Pisa_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '21_Pisa_S1.jpg']
Pisa_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '21_Pisa_S2.jpg']

Pisa_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '21b_Pisa_S1.jpg']
Pisa_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '21b_Pisa_S2.jpg']

Venedig_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '22_Venedig_S1.jpg']
Venedig_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '22_Venedig_S2.jpg']

Venedig_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '22b_Venedig_S1.jpg']
Venedig_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '22b_Venedig_S2.jpg']

Warschau_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '23_Warschau_S1.jpg']
Warschau_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '23_Warschau_S2.jpg']

Warschau_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '23b_Warschau_S1.jpg']
Warschau_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '23b_Warschau_S2.jpg']


Zürich_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '24_Zürich_S1.jpg']
Zürich_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '24_Zürich_S2.jpg']

Zürich_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '24b_Zürich_S1.jpg']
Zürich_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '24b_Zürich_S2.jpg']

In [6]:
#color palette
colors = Colorblind8
maps = Eyetracking_data['StimuliName'].unique().tolist()
maps.sort()

In [7]:
#Slider and drop down menu
slider_cluster = Slider(title="Amount of Clusters", start=1, end=8, value=2, step=1)
stimulimap = Select(title="Stimulimap", value=maps[0], options=maps)

In [8]:
#create source for AOI plot
source = ColumnDataSource(data=dict(x=[], y=[], c=[], u=[], color=[]))

In [9]:
#draw empty AOI plot
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"
TOOLTIPS=[("Participant", "@u"),
          ("AOI", "@c"),
         ("X-coordinate", "@x"),
         ("Y-coordinate", "@y")
         ]

p = figure(title="AOI plot", tools=TOOLS, tooltips=TOOLTIPS)
p.image_url(url=['01_Antwerpen_S1.jpg'], x=0, y=0, w=1650, h=1200) #not sure yet how to determine the width and hight
p.circle(x='x', y='y', color='color', legend_field='c', source=source, fill_alpha=0.2, size=10)
p.xgrid.grid_line_color = None
p.legend.orientation = "vertical"
p.legend.location = "bottom_right"
p.legend.title = 'AOI'
p.y_range.flipped = True

show(p)

In [10]:
#Create source for transition matrix
source_matrix = ColumnDataSource(data=dict(x1=[], y1=[], value=[]))

#Draw empty transition matrix figure
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"
TOOLTIPS=[("Value", "@value"),
         ("AOI transition from", "@x1"),
         ("to", "@y1")]

colormap = cm.get_cmap("BuPu")
bokehpalette = [mpl.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))]
mapper = LinearColorMapper(palette=bokehpalette, low=0.0, high=1.0)

z = figure(title="Transition Matrix", toolbar_location='below',
           toolbar_sticky=False, tools=TOOLS, tooltips=TOOLTIPS)
z.rect(x='x1', y='y1', width=1, height=1, source=source_matrix,
       fill_color={'field': 'value', 'transform': mapper}, line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="5pt",
                     ticker=BasicTicker(desired_num_ticks=8),
                     label_standoff=6, border_line_color=None, location=(0, 0))
z.add_layout(color_bar, 'right')
z.xaxis.axis_label = 'AOI'
z.yaxis.axis_label = 'AOI'

show(z)

In [11]:
#function calculating new dataframe based on slider value & selecting stimulus
def select_graphs():
    stimulimap_val = stimulimap.value
    selected = Eyetracking_data.copy()
    if (stimulimap_val != ""):
        selected = selected[selected['StimuliName'].str.contains(stimulimap_val)==True]
    X_km = selected[['MappedFixationPointX', 'MappedFixationPointY']].copy()
    km = KMeans(slider_cluster.value)
    km.fit(X_km)
    centers = pd.DataFrame(km.cluster_centers_, columns=X_km.columns)
    X_km['cluster'] = km.labels_

    user = selected["user"]
    X_km = X_km.join(user)
    X_km = X_km.reset_index()
    
    X_km_adj = X_km.copy()

    for i in range(X_km.index[-1]+1):
        X_km_adj.loc[i, 'cluster'] = X_km_adj['cluster'][i]+1

    return X_km_adj

In [12]:
#function calculating matrix dataframe based on slider value & selecting stimulus
def calc_matrix():
    stimulimap_val = stimulimap.value
    n_clusters = slider_cluster.value
    
    selected = Eyetracking_data.copy()
    if (stimulimap_val != ""):
        selected = selected[selected['StimuliName'].str.contains(stimulimap_val)==True]
    X_km = selected[['MappedFixationPointX', 'MappedFixationPointY']].copy()
    km = KMeans(n_clusters)
    km.fit(X_km)
    centers = pd.DataFrame(km.cluster_centers_, columns=X_km.columns)
    X_km['cluster'] = km.labels_

    user = selected["user"]
    X_km = X_km.join(user)
    X_km = X_km.reset_index()
    
    matrix = []
    AOI = []
    count = 0
    for i in range (0, n_clusters):
        matrix.append([])
        AOI.append(count+1)
        count += 1
    for i in range (0, n_clusters):
        for j in range (0, n_clusters):
            matrix[i].append(j)
            matrix[i][j] = 0
            
    cluster = X_km.loc[0, 'cluster']
    user = X_km.loc[0, 'user']
    for n in range(1, X_km.index[-1]+1):
        cluster_compare = X_km.loc[n, 'cluster']
        user_compare = X_km.loc[n, 'user']
        if cluster != cluster_compare and user == user_compare: #I think this solves the problem with the increment between users
            matrix[cluster][cluster_compare] = matrix[cluster][cluster_compare]+1
        cluster = cluster_compare
        user = user_compare
    
    matrix = np.array(matrix)
    
    m = np.amax(matrix)
    norm_matrix = (1/m) * matrix
    df_norm_matrix = pd.DataFrame(norm_matrix, index=AOI, columns=AOI)
    
    matrix_r = df_norm_matrix.reset_index()
    matrix_rows = pd.melt(matrix_r, id_vars=['index'], value_vars=AOI, var_name='target_AOI')

    return matrix_rows

In [13]:
#function updating column data source base on slider value & stimulus selection
def update():
    df = select_graphs()
    x = list(df['MappedFixationPointX'])
    y = list(df['MappedFixationPointY'])
    c = list(df['cluster'])
    u = list(df['user'])
    
    colorlist = []
    for i in range(df.index[-1]+1):
        index = df['cluster'][i]
        colorlist.append(colors[index])
    
    color = colorlist
    source.data = dict(
        x=x,
        y=y,
        c=c,
        u=u,
        color=color
    )
        
    #update matrix data source   
    matrix_rows = calc_matrix()
    x1 = list(matrix_rows['index'])
    y1 = list(matrix_rows['target_AOI'])
    value = list(matrix_rows['value'])
    
    source_matrix.data = dict(
        x1=x1,
        y1=y1,
        value=value
    )

In [14]:
#layout execution
controls = [slider_cluster, stimulimap]
for control in controls:
    control.on_change('value', lambda attr, old, new: update())

inputs = column(*controls, width=320, height=1000)
inputs.sizing_mode = "fixed"

l = layout([
    [inputs, p, z],
], sizing_mode="fixed")

update()  # initial load of the data

curdoc().add_root(l)
curdoc().title = "AOI plot"

In [15]:
show(l)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

